# Titanic_01 : 타이타닉 
---
- 타이타닉에 탑승한 사람들의 신상정보를 활용
- 승선한 사람들의 생존여부를 예측하는 모델을 생성
    - 여러 시각화 도구(matplotlib, seaborn, plotly), 데이터 분석 도구(pandas, numpy), 머신 러닝 도구(sklearn)을 사용

## 가설
---
1. Pclass :  좋은 객실을 사용한 승객이 더 많이 살아남았을 것이라는 가설 
2. Age : 젊은사람일수록 에너지가 있으니까 더 많이 살아 남았을거라는 가설 
3. Fare : 탑승료에 따른 생존률, 근데 이거는 pclass와 겹치거나 할것같음
4. sex : 여성이 더 많이 살아 남았을것이라는 가설 

In [1]:
# !pip install missingno 

In [2]:
# import
import numpy as np    # 넘파이
import pandas as pd   # 판다스 
import matplotlib.pyplot as plt   # 맷폴렛- 그래프
import seaborn as sns      # 시본 - 그래프 

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 
# 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.

import missingno as msno   # missingno =>  missingno 패키지를 사용하면 누락데이터에 대한 시각화


# ignore warnings
import warnings    # 워닝 방지
warnings.filterwarnings('ignore')

%matplotlib inline

## 앞으로의 프로세스
---

### 1. 데이터 셋 확인 
    - null data 확인(결측치)하고 추후 수정 
### 2. 탐색적 데이터 분석(**E**xploratory **D**ata **A**nalysis)  
    - 여러 특징(feature)들을 개별적으로 분석하고 특징들의 상관관계를 확인 
    - 여러 시각화 툴을 이용하여 통찰력(insight)을 얻음 
### 3. 특징을 엔지니어링
    - 모델의 성능을 높일 수 있도록 특징들을 엔지니어링
    - one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리등...
### 4. 모델 만들기 
    - 사잇킷런을 사용하여 모델을 생성 
    - sklearn 을 사용하면 수많은 알고리즘을 일괄된 문법으로 사용가능
### 5. 모델 학습 및 예측 
    - 트레인 셋을 가지고 모델 학습
    - 테스트 셋으로 예측(prediction)함 
### 6. 모델 평가 
    - 예측 성능이 원하는 수준인지 판단 
    - 풀려는 문제에 따라 모델을 평가하는 방식이 달라짐 

---

## 1. 데이터 셋 확인
---

- null data 확인하고 추후 수정 
- 판다스 => 테이블화 된 데이터를 다루는데 가장 최적화 된 라이브러리 
- pandas를 사용하여 데이터 셋의 간단한 통계적 분석, 복잡한 처리들을 간단한 메소드를 사용하여 작업  
- 판다스는 반복또 반복
---
- 캐글의 데이터 셋은 train, testset로 나누어짐 

In [13]:
def search_path(dir_name = 'help'):
    import os
    
# dirname = '../../input'
    txt_err = '''
    - 여기 참고!
    ex) 만약 input => C:\input\train.csv에 위치한다면,
    1.  ->  '/'    -> 가장 최상의 디렉토리로 이동
        => / : root('C:')
    2.  ->  './'   -> 파일이 현재 디렉토리를 의미    
        => ./ : now_dir -> 내위치(=train.csv) -> train.csv
    3.  -> '../'   -> 상위 디렉토리로 이동
        => ../ : now_dir의 상단 폴더  -> input
    - 두단계 상위 디렉토리로 이동하려면 =>  '../../' 이렇게 사용
    '''
    if dir_name == 'help':
        print(txt_err)
    else:
        for dirname, _, filenames in os.walk(dir_name):
            
        #     print(dirname, _, filenames)
            for filename in filenames:
#                 print(filename)
                print(os.path.join(dirname, filename))
    
search_path('../../input')

../../input\gender_submission.csv
../../input\test.csv
../../input\train.csv


In [10]:
path = '../../input/%s.csv' 

df_train = pd.read_csv(path %'train')
df_test  = pd.read_csv(path % 'test')


In [ ]:
df_train.head() # 데이터 확인 
# print(df_train.head())

In [ ]:
df_train.shape  # 학습용 데이터 크기 확인 

In [ ]:
df_test.shape # 테스트용 데이터 크기 확인 

In [ ]:
df_train.columns  # 컬넘 확인 

### 제공 데이터 확인 
---

- 데이터의 특징 
    - class, Age, SibSp, Parch, Fare
    
- 예측하고자 하는것은 생존자 
    - target label 은 Survived 
- parch : 함께 탑승한 부모, 아이의 수(정량적)
- SibSp : 함께 탑승한 형제와 배우자의 수(정략적)
- Fare  : 탑승료 

---
- describe() 메소드=> 특징을 가진 통계치들을 반환     
[describe()_Eng](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)

In [ ]:
# 학습 데이터 통계치 확인 
# describe()메소드 사용 
df_train.describe()# age에서 결측치 177개 확인가능 

# 컬럼이 전부타 나오는건 아닌가보다 => 수치화 데이터만 츨력

In [ ]:
df_test.describe()

In [ ]:
# 더확인 
df_train.isnull().sum()

In [ ]:
# 테스트 데이터 통계치 확인 
df_test.describe() # age에서 결측치 86 그리고 fare 결측치 1  확인 

In [ ]:
# 승객 아이디 숫자와 다른 결측데이터가 존재하는 열이 있음 
# 데이터를 한눈에 보기위해서 시각화 

# 결측치 확인 
---
- Null data check

In [ ]:
df_train.Age.isnull().sum()/ df_train['Age'].shape[0]

In [ ]:
# 틀 생성 
# 결측값의 비율을 한눈에 확인(퍼센트로)하기위해 만듬
for col in df_train.columns:
#     print(col)
    msg = 'column : {:>10}\t percent of NaN value: {:.2f}%'.format(col,100) # 퍼센트로 확인 
    print(msg)

In [ ]:
for col in df_train.columns:
    # percent ->  결측값을 비율화 시켜서 백분율로 
    # 비율 * 100 = 백분율 
    percent = df_train[col].isnull().sum() / df_train[col].shape[0]
#     print(percent)
    #결과
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
    print(msg)

In [ ]:
df_test.shape

In [ ]:
# test도 동일한 방법으로 확인 

for i in df_test.columns:
#     print(i) # 정상 출력 
    # 1. 일단 공식부터 예) 100개중 5개5개 2개 -> 5+5+2 /100 => 0.12  
    # => 결측치의 합 / 총 갯수 
    # shape => 
    percent = df_test[i].isnull().sum() / df_test[i].shape[0]
#     print(percent)
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
    print(msg)

In [ ]:
# train,test에서 Age(둘다 약 20%쯤), Cabin(둘다 약 80%쯤), Embarked(Train만 0.22%) 인것을 확인 
# index_loction => iloc[]
df_train.iloc[:,:]

### msno를 이용해서 결측치를 시각화
---

In [ ]:
# msno를 이용해서 결측치를 시각화
# 어느위치에 널데이터가 분포했는지를 확인 가능
msno.matrix(df = df_train.iloc[:,:],figsize=(8,8), color = (0.8,0.5,0.2))

In [ ]:
# 바차트  => 훨신 직관적 
# df_train
msno.bar(df = df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
# df_test
msno.bar(df = df_test.iloc[:,:], figsize=(8,8), color=(1, 0.5, 0.6))

## 타겟 데이터 확인 
---
- Target label cheak
    - 어떤 분포(distribution)를 가지고 있는 지 확인이 필요하다.
    - 이진 분류(binary classification)문제에서는 0과 1의 분포가 어떤지에 따라 모델의 평가 방법이 다르다.
    - 타겟 레이블의 목적성이 명확해야한다. 



#### 그래프를 그려보겠음 
---

In [ ]:
df_train['Survived'].value_counts() # 0: 죽은사람 1: 산사람

In [ ]:
# 총 인원 
df_train['Survived'].value_counts().sum()

In [ ]:
# 시리즈들은 항상 plot을 가지고 있음
# plt.plot(df_train['Survived'].value_counts())과 동일 
df_train['Survived'].value_counts().plot()

In [ ]:
# 판다스가 메소드 형식으로 그래프 그리는 함수를 통일화시켜둠 
df_train['Survived'].value_counts().plot.pie()

In [ ]:
# pie에서 -> explode 기능을 확인 => 째는거 
df_train['Survived'].value_counts().plot.pie(explode=[0,0.1])

In [ ]:
# pie에서 -> autopct 기능을 확인 =>  그래프 안에 퍼센트로 수치를 표현 
df_train['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct ="%1.1f%%")

In [ ]:
# pie에서 ->  shadow 기능을 확인 =>  True :그림자를 만든다.
df_train['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct ="%1.1f%%", shadow =True)

In [ ]:
# 그래프 => 두개를 그림 
# 하나는 원 그래프 하나는 막대 그래프 

f, ax = plt.subplots(1,2,figsize = (18,8))

# 1번 : pieplot
df_train['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct ="%1.1f%%",ax =ax[0], shadow =True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('') # 왼쪽의 레이블을 없애겠다

# 2번 : countplot
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()


- 죽은 사람이 눈에 띄게 많은 것을 확인할 수 있음
- 38.4% 만 살아 남은것을 확인할 수 있음 

---
# 탐색적 데이터 분석(EDA - Exploratory Data Analysis )
---
- 분석을 통해서 데이터 안에 숨겨진 정보를 찾아야한다.
- 시각화 라이브러리를 이용해 정보들을 시각화
    - 시각화 팩 : matplotlib, seaborn, plotly


----
- 목적을 찾아야 한다 
- 내가 하고자하는말 내가 보고자하는것을 시각화하는 능력을 키워야한다.
---

## Pclass
---
- pclass는 서수형 데이터 
    -> 카테고리형태면서 순서를 나타내는 타입 
> 카테고리하면 떠올라야하는 것 -> **원핫 벡터**

- 일단 pclass에 따른 생존률 차이를 확인 
    - 엑셀의 피벗차트와 유사한 작업이 진행 할 것이다.
    - 판다스의 데이터 프레임에서 groupby를 이용하면 동일한 기능을 쉽게 사용 가능하다.
    
- 'Pclass', 'Survived' 를 palss로 묶는다. 
    - 각 pcalss 마다 0,1이 count => 평균을 내면 각 pclass 별로 생존률을 확인할 수 있다.
    
---


#### Pclass의 가설 
---

- Pclass 
    => 좋은 객실을 사용한 승객이 더 많이 살아남았을 것이라는 가설 
    => 이 의문을 EDA를 통해서 검증하고자 한다. 

In [ ]:
# sum과 count의 차이 => 
df_train['Survived'].unique() # 0 과 1로 이루어 져있음 

In [ ]:
# 0과 1의 (객체) 수를 세하는것 
# df_train[['Pclass','Survived']].groupby(['Pclass']).count()
# 0과1을 연산하는것 
df_train[['Pclass','Survived']].groupby(['Pclass']).sum()

In [ ]:
# 데이터 프레임에 여러 컬럼이 있을 때 리스트로 컬럼을 묶음
# df_train[['Pclass','Survived']]
# df_train[['Pclass','Survived']].groupby
# df_train[['Pclass','Survived']].groupby(['Pclass']) # DataFrameGroupBy 객체가 만들어짐 
# 객체는 많은것을 가지고 있음 ..sum , count 등등 
df_train[['Pclass','Survived']].groupby(['Pclass']).count()

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'], as_index = True).count()

In [ ]:
type(df_train[['Pclass','Survived']].groupby(['Pclass'], as_index = True).count())

In [ ]:
# crosstab
# 쉽게 확인 가능 
# all의 여부  =>  margins= True
# 스타일 추가 => .style.background_gradient(cmap='summer_r') # GnBu # cool
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins= True).style.background_gradient(cmap='GnBu')

In [ ]:
# groupby에 mean() => 클래스 생존률을 얻을수 있음 80 / 80+136
class_Survived = df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).mean()
class_Survived

In [ ]:
# ascending : 차순을 어떻게 할건지 
class_Survived.sort_values(by='Survived', ascending= False ).plot.bar()

- 클래스가 좋을수록 생존 확률이 높음 
- seaborn 의 countplot을 이용 -> 특정 레이블에 따른 개수를 확인 

In [ ]:
y_position = 1.02

f, ax = plt.subplots(1,2,figsize = (18,8))
df_train['Pclass'].value_counts().plot.bar( color = ['#CD7F32','#FFDF00','#D3D3D3'], ax = ax[0])
ax[0].set_title('Number of Passengers By Pclass', y = y_position )
ax[0].set_ylabel('Count')

# df_train 
# hue = 'Survived' # 바 색 설명
sns.countplot('Pclass',hue = 'Survived', data = df_train, ax = ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y = y_position )

plt.show()

# 뭐를 보여주기 위해서 이 그래프를 그렸는지 잘 봐야한다. 
# 분석 해보자면 클래스별로 탑승인원을 그래프로 보여준다. 
# 그리고 이 탑승 인원별 생존자를 다시 그래프로 그린다. 

# 결론 ; 클래스가 좋을 경우 더 많이 생존한 것을 확인 가능하다.

---

## Sex 
---

- 기준은 생존
- 성별로 생존률이 어떻게 달라지는 지 확인 
- 위와 같이 pandas와 seaborn 사용

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (18,8))
# 1번 그래프
df_train[['Sex','Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
# 2번 그래프 
sns.countplot('Sex' ,hue='Survived', data=df_train,ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')

plt.show()

# 그래프를 통해 여자가 더 생존률이 높다는 것을 알 수 있다.

In [ ]:
# 성별을 기준으로 그룹바이 
df_train[['Sex','Survived']].groupby(['Sex'],as_index=False).mean()

In [ ]:
df_groupby = df_train[['Sex','Survived']].groupby(['Sex'],as_index=False).mean()
df_groupby.sort_values(by='Survived', ascending=False)

In [ ]:
# 위에서 한번했음 
# pands crosstab
pd.crosstab(df_train['Sex'],df_train['Survived'], margins= True)

In [ ]:
# 보기 편하게 스타일을 입히겠음 
df_crosstab = pd.crosstab(df_train['Sex'],df_train['Survived'], margins= True)
df_crosstab.style.background_gradient(cmap='summer_r') # 'GnBu'

- pclass와 마찬가지ㅡ  성별도 예측 모델에 쓰일 중요한 특징

---

## Both Sex and Pclass
---

- Sex and Pclass 두가지에 관해서 생존이 어떻게 연관이 있는지 확인 
- seaborn 의 factorplot 을 이용

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex', data = df_train , size=5 , aspect=1.5)

- 3가지를 한그래프에서 비교 ... 워우 
- 모든 클래스에서 female이 살 확률이 male보다 높은것을 확인 
- 남,여 상관없이 클래스가 높을수록 살 확률이 높음 

In [ ]:
# hue 대신 column
# 각각 출력 
sns.factorplot(x='Sex',y='Survived', col='Pclass',data=df_train, satureation=.5, sixz=10 , aspect=1)

---

## Age
---

- 나이의 특징을 확인 
- 살아있는 승객만 추출 

In [ ]:
# 나이의 특징을 확인 
# {:.1f} => 소수점 한자리까지 표현 => 데이터와 데이터 통일 
# print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train.['Age'].max())) 
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train.Age.max())) 
# print('제일 나이 어린 탑승객 : {:.1f} Years'.format(df_train.['Age'].min()))
print('제일 나이 어린 탑승객 : {:.1f} Years'.format(df_train.Age.min()))
print('탑승객 평균 나이: {:.1f} Years'.format(df_train.Age.mean()))

In [ ]:
# 살아있는 승객만 추출 
df_train[df_train['Survived'] == 1 ].head()

In [ ]:
# 살아있는 승객이 나이만 추출 
df_train[df_train['Survived'] == 1 ]['Age']

In [ ]:
# 생존에 따른 Age 히스토그램 
# 비교 -> 산사람 나이랑 죽은사람 나이를 그래프를 통해 비교 
fig, ax = plt.subplots(1,1, figsize = (9, 5)) 
sns.kdeplot(df_train[df_train['Survived'] == 1 ]['Age'], ax = ax )
sns.kdeplot(df_train[df_train['Survived'] == 0 ]['Age'], ax = ax )

plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

# 생존자 중 나이가 어린 경우가 많은것을 볼 수 있음 

In [ ]:
# 나이에 따른 클래스  
first_calss  = df_train['Age'][df_train['Pclass'] == 1]
second_calss = df_train['Age'][df_train['Pclass'] == 2]
third_calss  = df_train['Age'][df_train['Pclass'] == 3]
# print(first)

In [ ]:
# 클래스안 나이의 분포 확인 
plt.figure(figsize=(8,6))

# 그래프 그리기 
first_calss.plot(kind='kde')
second_calss.plot(kind='kde')
third_calss.plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
# 이름표
plt.legend(['1st Class','2nd Class','3rd Class'])

# class 가 높을수록 나이 많은 사람의 비중이 커짐 
# 나이대가 변하면서 생존률이 어떻게 되는지 확인이 필요 
# 나이의 범위를 점점 넓혀가면서 생존률 확인 

In [ ]:
cummulate_survival_ratio = []

for i in range(1,80):
    # 평균을 구하네 
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum()/ len(df_train[df_train['Age']< i ]['Survived']))
    
# cummulate_survival_ratio

In [ ]:
cummulate_survival_ratio = []

for i in range(1,80):
    # 평균을 구하네 
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum()/ len(df_train[df_train['Age']< i ]['Survived']))
    
# cummulate_survival_ratio

plt.figure(figsize=(7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age',y=1.02)

plt.xlabel('Range of Age(0~x)')
plt.ylabel('Survival rate')

plt.show()

# 나이가 어릴수록 생존률이 높다는것을 확인 가능 
# 나이가 중요한 특징임을 확인 

---

## Pclass, Sex , Age 
---

- 위 세가지르 한그래프에 담아 확인 => seaborn의 바이올린 폴렛을 이용해서 작업 
- 나눠서 확인 : x축은 case(Pclass, Sex)를 y축은 distribution(Age)

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (18, 8))

sns.violinplot("Pclass","Age", hue= "Survived", data = df_train, scale='count', split = True, ax = ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex", "Age", hue= "Survived", data = df_train, scale ='count', split = True, ax = ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

- 클래스 별로 각 아이템의 분포가 어떻게 다른지 거기에 생존여부에 따라 구분한 그래프 
- 생존만 봤을때 모든 클래스에서 나이가 어릴수록 많이 생존한것을 확인 
- 성별로 봤을 경우 남성보다 여성이 많이 생존한 것을 확인 

- 침몰때 여성과 아이를 먼저 챙긴 것을 볼수 있음 

---

## Embarked (탑승 항구)
---

- 탑승한 곳에 따른 생존률 

In [ ]:
# 그룹화 해서 평균 
embarked_group= df_train[['Embarked','Survived']].groupby(['Embarked'], as_index=True)
embarked_group.mean()

In [ ]:
f, ax = plt.subplots(1,1,figsize=(7,7))

embarked_group.mean().sort_values(by='Survived',ascending=False).plot.bar(ax=ax)

- 생존률은 비슷한것으로 확인 
- 명확하지 않지만 모델에 사용 
- 모델을 만들고나면 각 특징이 얼마나 중요한 역활을 했는지를 확인 가능 

- 다른 특징으로 split해서 확인 

In [ ]:
f, ax = plt.subplots(2,2, figsize=(20,15))

# 탑승항구에 따른 승객 수 
sns.countplot('Embarked', data=df_train, ax = ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')
# 탑승 항구에 따른 성별
sns.countplot('Embarked', hue= 'Sex', data=df_train, ax = ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
# 탑승항구에 따른 생존률 
sns.countplot('Embarked', hue='Survived', data=df_train, ax = ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
# 탑승항구에 따른 클래스 
sns.countplot('Embarked', hue='Pclass', data = df_train, ax = ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2, hspace=0.5) # 각 그래프간 간격 조절 
plt.show()

- 1. 전체적으로 S에서 탑승자수가 가장 많은것으로 확인 
- 2. C와Q는 남녀의 비율이 비슷하고, S는 남자가 더 많은것으로 확인 
- 3. 생존확률이 S경우 많이 낮은것을 확인 할수 있음 
- 4. 클래스로 나누어보니 C가 생존할 확률이 높은건 사람이 많이 탑승했기 때문 S는 3클래스 사람들이 적게 타서 생존률이 낮은것을 확인 


---

## Family - SibSp(형제, 자매) + Parch(부모, 자녀)
---

- Family =  SibSp + Parch

In [ ]:
df_train['SibSp'] + df_train['Parch']

In [ ]:
# 자신이 미포한 그래서 1을 더해줌 
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

In [ ]:
# 최대
print("Maximum size of Family:",df_train['FamilySize'].max())

# 최소
print("Maximum size of Family:",df_train['FamilySize'].min())

In [ ]:
df_train[['FamilySize','Survived']].groupby(['FamilySize'],as_index =True).mean()

In [ ]:
grooup_Fam = df_train[['FamilySize','Survived']].groupby(['FamilySize'],as_index =True)
grooup_Fam.mean().sort_values(by='Survived',ascending= False)

In [ ]:
# 가족수와 생존의 상관 관계를 확인 
f, ax = plt.subplots(1,3, figsize= (40,10))

sns.countplot('FamilySize', data= df_train, ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot('FamilySize', hue='Survived',data = df_train, ax= ax[1])
# y=1.02 는 글자와 그래프 사이의 간격 조절 
ax[1].set_title('(2) Survived countplot depending on FamilySize',y=1.02)

grooup_Fam.mean().sort_values(by='Survived',ascending= False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize',  y=1.02 )

plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

- 1. 가족의 크기가 1명에서 11명 까지 있다는것을 확인 
    - 대부분 1명
- 2,3. 가족의 크기에 따른 생존률 비교 
    - 4인 가족인경우가 생존률이 가장 높음
    - 5인 이상부터 생존률이 하락하는 것을 확인
    - 가족수가 작아도 커도 생존확률이 적은것을 확인 
    - 3~4명선이 가장 생존률이 많은 것을 확인 
    

---

## Fare
---
- Fare는 탑승요금인 동시에 contious feature
- histogram 표현해보자

In [ ]:
fig, ax = plt.subplots( 1, 1, figsize=(8,8) )

g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness :{:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g =g.legend(loc='best')

- distribution(분포)가 매우 비대칭인 것을 알 수 있다. 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있다. 
- outlier에 대해 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있다. 그렇기 때문에 outlier의 영향을 줄이기위해 탑승요금(fare)에 log처리 하겠다.


- 원하는 데이터는 탑습요금 컬럼의 데이터 모두 로그처리하는것
    - 판다스의 람다함수를 이용 
        - 데이터 프레임의 특정 컬럼에 공통된 작업을 적용
          - 람다함수를 이용해 map함수에 인수에 할당하면 탑승요금 컬럼에 바로 적용

[데이터 프레임 고급인덱싱_loc](https://datascienceschool.net/view-notebook/704731b41f794b8ea00768f5b0904512/) 매번 헷갈림

In [ ]:
# 사본으로 생성 - Fare 에 있는 넌값 추출 
df_test.loc[df_test.Fare.isnull(),'Fare'] = df_test['Fare'].mean()
# 테스트 셋에 있는 결측값을 평균값으로 치환 **

In [ ]:
# 13번강의 들을것 

In [ ]:
# 람다함수를 이용해 fare에 값을 입력 
df_train['Fare'] = df_train['Fare'].map(lambda i : np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i : np.log(i) if i > 0 else 0)


In [ ]:
# 그래프 그리기 
fig, ax = plt.subplots(1,1, figsize =(10,8))
g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax = ax)
g = g.legend(loc = 'best')

- 로그를 씌우니, 비대칭이었던 fare의 비대칭성이 많이 사라진것을 확인 가능 
- 이런 작업들을 통해서 모델을 좀 더 좋은 성능을 내도록 할 수 있다.
- 위에서 한부분은 feature engineering 에 들어가는 부분인데, 여기서 작업을 진행되었다.
- **모델을 학습시키기 위해, 그리고 그모델의 성능을 높이기 위해 특징들에 여러 조작을 가하거나, 새로운 특징을 추가하는것을 feature engineering이라고 함**

---

## Cabin
---

- 이 특징은 결측값이 약 80%
    - 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지 않다.
- 그렇기 때문에 만들고자 하는 모델에 포함시키지 않겠다. 

In [ ]:
df_train.head()

## Ticket
---

- 티켓의 feature은 결측치가 없다. 
- 이 데이터는 string data(문자열)이기 때문에 우리가 사용할 모델에 직접적으로 사용 할 수없고 가공을 해야 사용이 가능하다. 
- 가공을 하기위해서 어떻게 가공할건지에 대한 아이디어가 필요하다.

In [ ]:
# 중복 데이터 카운드 
df_train.Ticket.value_counts()

- 티켓 번호는 정말 다양한것을 확인 
    - 여기서 어떤 특징을 끌어내야 생존과 연결이 가능할것인가? 
    - 아이디어 필요 ... 
    - 지금은 넘어가지만 모델을 향상하고 싶으면 이티켓의 특징을 살리는것 또한 한 방법